In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

print('--start--')
time_start = time.time()

df_train = pd.read_csv('gs://sample_machine_learning_input/HousePrices/train.csv')
df_test = pd.read_csv('gs://sample_machine_learning_input/HousePrices/test.csv')

print('--1--')

# データタイプがobjectの列の値をラベル化した数値に変換
for i in range(df_train.shape[1]):
    if df_train.iloc[:,i].dtypes == object:
        lbl = LabelEncoder()
        lbl.fit(list(df_train.iloc[:,i].values) + list(df_test.iloc[:,i].values))
        df_train.iloc[:,i] = lbl.transform(list(df_train.iloc[:,i].values))
        df_test.iloc[:,i] = lbl.transform(list(df_test.iloc[:,i].values))


print('--2--')
#外れデータ除外
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)
df_train = df_train.drop(df_train[(df_train['LotArea']>100000)].index)
df_train = df_train.drop(df_train[(df_train['Street']<0.1)].index)
df_train = df_train.drop(df_train[(df_train['Utilities']>0.9)].index)
df_train = df_train.drop(df_train[(df_train['SalePrice']>700000)].index)
df_train = df_train.drop(df_train[(df_train['Electrical']>4.5)].index)
df_train = df_train.drop(df_train[(df_train['LowQualFinSF']>560)].index)
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4500)].index)
df_train = df_train.drop(df_train[(df_train['BsmtFullBath']>2.5)].index)
df_train = df_train.drop(df_train[(df_train['BsmtHalfBath']>1.75)].index)
df_train = df_train.drop(df_train[(df_train['BedroomAbvGr']>7)].index)
df_train = df_train.drop(df_train[(df_train['KitchenAbvGr']>2.75)].index)
df_train = df_train.drop(df_train[(df_train['OpenPorchSF']>500)].index)
df_train = df_train.drop(df_train[(df_train['EnclosedPorch']>500)].index)
df_train = df_train.drop(df_train[(df_train['SaleCondition']>-1) & (df_train['SalePrice']>700000)].index)




print('--3--')

Xmat = pd.concat([df_train, df_test])
# 欠損が多いカラムを削除
Xmat = Xmat.drop(['LotFrontage','MasVnrArea','GarageYrBlt'], axis=1)
Xmat = Xmat.drop(['PoolArea','PoolQC','MoSold','YrSold'], axis=1)
Xmat = Xmat.drop(['Fireplaces','FireplaceQu'], axis=1)
Xmat = Xmat.drop(['Street','Utilities','Condition2','BsmtHalfBath','3SsnPorch','MiscVal'], axis=1)
Xmat = Xmat.drop(['LotShape','LotConfig','BsmtFinSF1','ScreenPorch'], axis=1)
Xmat['TotalSF'] = Xmat['TotalBsmtSF'] + Xmat['1stFlrSF'] + Xmat['2ndFlrSF']

# 欠損値の少ないカラムのNaNは中央値(median)で埋める
Xmat = Xmat.fillna(Xmat.median())

X_ch_train = Xmat.iloc[:df_train.shape[0],:]
X_ch_test = Xmat.iloc[df_train.shape[0]:,:]



print('--4--')
#
test_id = X_ch_test['Id']
df_train_y = X_ch_train['SalePrice']
df_test_id = X_ch_test['Id']

df_train_x = X_ch_train.drop(['Id','SalePrice'],axis=1)
X_ch_test = X_ch_test.drop(['Id','SalePrice'],axis=1)

y_train_log = np.log(df_train_y)


print(df_train_x.shape)
print(X_ch_test.shape)

'''
X_train, X_test, y_train, y_test = train_test_split(df_train_x, df_train_y, random_state=0)

for alpha in [0.00001,0.0001,0.001,0.01, 0.1, 1, 3, 7, 10, 100,200, 300, 400 ,500,600, 1000]:
    ridge = Ridge(alpha=alpha).fit(X_train, y_train)
    #print("\n alpha={}".format(str(alpha)))
    #print("Train set score: {:.2f}".format(ridge.score(X_train, y_train)))
    #print("Test set score: {:.2f}".format(ridge.score(X_test, y_test)))


for alpha in [0.00001,0.0001,0.001,0.01,0.02, 0.5, 1,10,100,200, 300, 400 ,500,600, 1000,5000,10000]:
    lasso = Lasso(alpha=alpha).fit(X_train, y_train)
    #print("\n alpha={}".format(str(alpha)))
    #print("Train set score: {:.2f}".format(lasso.score(X_train, y_train)))
    #print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))
    #print("Number of features used:{}".format(np.sum(lasso.coef_ != 0)))
    
for val_max_depth in [4,5,6]:
    for val_n_estimators in [10,100,200,300,500,1000]:
        sgr = GradientBoostingRegressor(max_depth=val_max_depth, n_estimators=val_n_estimators)
        sgr_data = sgr.fit(df_train_x,df_train_y)    
        #print("Test set score: {:.2f}".format(sgr_data.score(X_test, y_test)))
    
'''


print('-------Linear')
lin_model = LinearRegression()
lin_model_data = lin_model.fit(df_train_x,y_train_log)
print(lin_model.score(df_train_x,y_train_log))

print('-------Ridge')
ridge_model = Ridge(alpha=0.000001)
ridge_model_data = ridge_model.fit(df_train_x,y_train_log)
print(ridge_model.score(df_train_x,y_train_log))

print('-------Lasso')
#clf_model = Lasso(alpha=0.000001) 
#clf_model_data = clf_model.fit(df_train_x,y_train_log)
#print(clf_model.score(df_train_x,y_train_log))
clf_model_data = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(df_train_x,y_train_log)


print('-------xgboost')
xgb_model = xgb.XGBRegressor(n_estimators=1000, max_depth=2, learning_rate=0.1)
xgb_model_data=xgb_model.fit(df_train_x,y_train_log)

'''
#xgb_model = xgb.XGBRegressor()
xgboost = xgb.XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006, random_state=42)
'''

print('-------GradientBoostingRegressor')
sgr = GradientBoostingRegressor(max_depth=6, n_estimators=1000)
sgr_data = sgr.fit(df_train_x,y_train_log)
print(sgr_data.score(df_train_x,y_train_log))
'''
#score:0.12050
blend_models_predict = ((0.4 * lin_model_data.predict(X_ch_test)) 
                        + (0.1 * ridge_model_data.predict(X_ch_test)) 
                        + (0.05 * clf_model_data.predict(X_ch_test))
                        + (0.4 * xgb_model_data.predict(X_ch_test))
                        + (0.05 * sgr_data.predict(X_ch_test)))
'''
'''
#score:0.12039
blend_models_predict = ((0.4 * lin_model_data.predict(X_ch_test)) 
                        + (0.6 * xgb_model_data.predict(X_ch_test)))
'''
#0.12021
blend_models_predict = ((0.3 * lin_model_data.predict(X_ch_test)) 
                        + (0.05 * ridge_model_data.predict(X_ch_test)) 
                        + (0.05 * clf_model_data.predict(X_ch_test))
                        + (0.5 * xgb_model_data.predict(X_ch_test)))


y_test_pred = np.exp(blend_models_predict)

submission = pd.DataFrame({
    "Id": test_id,
    "SalePrice": y_test_pred
})
submission.to_csv('gs://sample_machine_learning_output/HousePrices/hp_submission10.csv', index=False)
########

print('--end--',time.time()-time_start)

--start--
--1--
--2--
--3--
--4--
(1434, 61)
(1459, 61)
-------Linear
0.9176407767301069
-------Ridge
0.9176407767301068
-------Lasso
-------xgboost
-------GradientBoostingRegressor
0.9999922420329622
(1459, 61)
--end-- 11.02367091178894
